# Étude de cas : restaurant Chipotle

## Se renseigner sur ce qu’est un KPI et à quoi cela sert dans le domaine de la data analyse 

Wikipedia : *Un indicateur clé de performance (ICP) – en anglais : key performance indicator (KPI) – est un indicateur utilisé pour l'aide à la décision dans les organisations. Les ICP sont utilisés particulièrement en gestion de la performance organisationnelle.*

En d'autres termes, c'est un nombre que l'on peut associer à un certain jeu de données, réel ou projeté en fonction des décisions prises, qui permet de mesurer si la situation correspondante est bonne ou mauvaise. 
Dans le cas d'un restaurant, ce serait avant tout les profits. Mais on pourrait aussi utiliser par exemple le nombre de clients ou de commandes pour une analyse plus poussée, étant donné qu'un même profit pour un petit ou un grand nombre de clients représenteraient des stratégies différentes.

## Importez les bonnes librairies et le dataset resto.csv

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
data=pd.read_csv('./data/resto.csv')

## Analyser les differentes variables (columns) du dataset

In [3]:
print(data.dtypes)
data.head()

Unnamed: 0             int64
order_id               int64
quantity               int64
item_name             object
choice_description    object
item_price            object
dtype: object


,Unnamed: 0,order_id,quantity,item_name,choice_description,item_price
0,0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,1,Izze,[Clementine],$3.39
2,2,1,1,Nantucket Nectar,[Apple],$3.39
3,3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


Les données contiennent 4621 échantillons avec 6 caractéristiques, représentant les plats commandés pendant une période indéterminée, probablement une journée : 
- 'Unnamed: 0' : un index redondant des plats commandés
- 'order_id' : variable discrète de la commande correspondant à un ensemble de plats
- 'quantity' : la quantité de chaque plat 
- 'item_name' : le nom du plat
- 'choice_description' : un descriptif supplémentaire pour les plats disponibles sous plusieurs formes, que j'appellerai une option.
- 'price' : le prix du plat. On notera que c'est une chaine de caractères en raison du symbole $.

In [4]:
#On regarde si le prix est unitaire ou total :
print(data.loc[ (data['item_name'] == data.loc[0,'item_name']) 
              & (data['quantity'] != data.loc[0,'quantity'])  , ('quantity', 'item_price') ] )

      quantity item_price
1257         4    $11.80 
1882         2     $5.90 
2267         2     $5.90 
2729         2     $5.90 
3598        15    $44.25 


In [5]:
#On regarde pour un 'item_name' sujet à de fortes variations si 'choice_description' affecte le prix : 
print(data.loc[ (data['item_name'] == data.loc[5,'item_name']) 
          & (data['quantity'] == data.loc[5,'quantity']) 
          & (data['choice_description'] != data.loc[5,'choice_description']) 
          , ('item_name', 'choice_description', 'item_price')])

         item_name                                 choice_description  \
13    Chicken Bowl  [Fresh Tomato Salsa, [Fajita Vegetables, Rice,...   
19    Chicken Bowl  [Tomatillo Red Chili Salsa, [Fajita Vegetables...   
26    Chicken Bowl  [Roasted Chili Corn Salsa (Medium), [Pinto Bea...   
42    Chicken Bowl  [Roasted Chili Corn Salsa, [Rice, Black Beans,...   
76    Chicken Bowl  [Fresh Tomato Salsa, [Rice, Black Beans, Pinto...   
...            ...                                                ...   
4590  Chicken Bowl  [Roasted Chili Corn Salsa, [Rice, Black Beans,...   
4591  Chicken Bowl  [Tomatillo Red Chili Salsa, [Rice, Black Beans...   
4595  Chicken Bowl  [Tomatillo Green Chili Salsa, [Rice, Black Bea...   
4599  Chicken Bowl      [Roasted Chili Corn Salsa, [Cheese, Lettuce]]   
4604  Chicken Bowl  [Fresh Tomato Salsa, [Rice, Black Beans, Chees...   

     item_price  
13      $11.25   
19       $8.75   
26       $8.49   
42      $11.25   
76       $8.75   
...         ...

On peut voir que le prix est un prix total, et qu'il varie selon les options du plat.

## Afficher la dimension du dataset et les statistique descriptives

In [6]:
data.describe(include='all')

,Unnamed: 0,order_id,quantity,item_name,choice_description,item_price
count,4622.000000,4622.000000,4622.000000,4622,3376,4622
unique,NaN,NaN,NaN,50,1043,78
top,NaN,NaN,NaN,Chicken Bowl,[Diet Coke],$8.75
freq,NaN,NaN,NaN,726,134,730
mean,2310.500000,927.254868,1.075725,NaN,NaN,NaN
std,1334.400802,528.890796,0.410186,NaN,NaN,NaN
min,0.000000,1.000000,1.000000,NaN,NaN,NaN
25%,1155.250000,477.250000,1.000000,NaN,NaN,NaN
50%,2310.500000,926.000000,1.000000,NaN,NaN,NaN
75%,3465.750000,1393.000000,1.000000,NaN,NaN,NaN


##  Afficher toutes les colonnes du dataset sous forme d'objet list python

In [7]:
#Si la question veut dire afficher l'ensemble des intitulés des colonnes comme une liste : 
cols=data.columns.to_list()
print(cols)

['Unnamed: 0', 'order_id', 'quantity', 'item_name', 'choice_description', 'item_price']


In [8]:
#Si la question veut dire afficher le contenu des colonnes dans des listes :
for i in cols :
#     print(set(data[i].to_list()))
    print(data[i].to_list())
    print('\n')


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

## Quel est l’item le plus commandé ?

On récupère un objet qui pour chaque nom de plat contient les quantités commandées. La fonction somme s'applique ensuite à chaque groupe pour sommer ces quantités.
On peut alors simplement afficher la valeur la plus élevée.

In [9]:
print(data.groupby('item_name').quantity.sum().nlargest(1))

item_name
Chicken Bowl    761
Name: quantity, dtype: int64


On peut aussi afficher une liste triée pour voir un peu les autres éléments. 

Cela permet d'ailleurs de voir que certains éléments avec un tiret sont en double. On pourrait éditer les 'item_name' pour remplacer les tirets par un espace, mais ce regroupement ne changerait pas l'élément le plus commandé de toutes façons.

In [10]:
print(data.groupby('item_name').quantity.sum().sort_values(ascending=False))

item_name
Chicken Bowl                             761
Chicken Burrito                          591
Chips and Guacamole                      506
Steak Burrito                            386
Canned Soft Drink                        351
Chips                                    230
Steak Bowl                               221
Bottled Water                            211
Chips and Fresh Tomato Salsa             130
Canned Soda                              126
Chicken Salad Bowl                       123
Chicken Soft Tacos                       120
Side of Chips                            110
Veggie Burrito                            97
Barbacoa Burrito                          91
Veggie Bowl                               87
Carnitas Bowl                             71
Barbacoa Bowl                             66
Carnitas Burrito                          60
Steak Soft Tacos                          56
6 Pack Soft Drink                         55
Chips and Tomatillo Red Chili Salsa       50


## Combien d’items ont été commandés au total chez Chipotle

Il suffit de sommer les quantités de commandes.

In [11]:
print(data.quantity.sum())

4972


## Combien de chiffre d’affaire à realisé la société ?

On avait vu en introduction que les prix sont des chaines de caractères à cause du symbole dollar. On doit donc retirer ce symbole puis les convertir en float avant de sommer :

In [12]:
liste_prix=[float(st.replace('$', '')) for st in data.item_price]

In [13]:
print(np.sum(liste_prix))

34500.16


## Quel est le revenu moyen par commande ? 

On a le revenu total. Il faut donc obtenir le nombre de commandes.

In [14]:
print( len(data.order_id.unique()) )

1834


In [15]:
print(f"Le revenu moyen est de {np.sum(liste_prix) / len(data.order_id.unique()):.2f}$" )

Le revenu moyen est de 18.81$


## Formaliser une petite conclusion sous forme de rapport d'analyse dans un jupyter notebook en ajoutant si vous le voulez des graphiques ou d'autres analyses pertinentes. 